In [1]:
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
import os
import pandas as pd
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from pathlib import Path
from amftrack.pipeline.launching.run_super import run_parallel



/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/gpfs/scratch1/shared/amftrackflow/temp


2023-05-04 11:55:16.944133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 11:55:17.223728: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home6/svstaalduine/.local/lib/python3.9/site-packages/cv2/../../lib64:/sw/arch/Centos8/EB_production/2021/software/ZeroMQ/4.3.4-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/util-linux/2.36-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/libsodium/1.0.18-GCCcore-10.3.0/lib:/sw/arch/Centos8/EB_production/2021/software/OpenPGM/5.2.122-GCCcore-10.3.0/lib:/sw

In [51]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20221108_Plate462/"


In [52]:

plate_list = glob(directory_targ + "*_Plate*_*")
datadict = {'address_total' : plate_list}
dataframe = pd.DataFrame(data=datadict)
dataframe['video_name'] = [address.split('/')[-1] for address in dataframe['address_total']]
dataframe['plate_nr'] = [address.split('/')[-1].split('_')[-2][5:] for address in dataframe['address_total']]
dataframe['video_nr'] = [int(address.split('/')[-1].split('_')[-1]) for address in dataframe['address_total']]
dataframe['date_imaged'] = [address.split('/')[-1][:8] for address in dataframe['address_total']]

parent_folder = []
data_table = []
magnification = []
fps = []
mode = []

for row in dataframe.iloc:
#     print(row)
    parent_folder.append(str(Path(row['address_total']).parent))
    excel_file = glob(str(Path(row['address_total']).parent) + f'/*{row["date_imaged"]}*{row["plate_nr"]}.xl*')
    if len(excel_file) > 0:
        data_table.append(excel_file[0])
        excel_table = pd.read_excel(excel_file[0])
#         print(row['video_name'])
#         print(excel_table['Unnamed: 0'][21])
#         print(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'])
        magnification.append(excel_table[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Magnification'].iloc[0])
        fps.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['FPS'].iloc[0])
        mode.append(excel_table.loc[excel_table['Unnamed: 0'].str.contains(row['video_name'], case=False, na=False)]['Bright-field (BF)\nor\nFluorescence (F)'].iloc[0])
        
    else:
        csv_file = glob(str(Path(row['address_total']).parent) + f'/{row["date_imaged"]}*{row["plate_nr"]}.csv')
        if len(csv_file) > 0:
            data_table.append(csv_file[0])
        else:
            print("Halp! No datatable found!")

dataframe['fps'] = fps
dataframe['magnification'] = magnification
dataframe['mode'] = mode
dataframe['parent_folder'] = parent_folder
dataframe['data_table'] = data_table

dataframe = dataframe.sort_values(by='video_name')
dataframe = dataframe[dataframe['video_nr'].le(4)]
dataframe


,address_total,video_name,plate_nr,video_nr,date_imaged,fps,magnification,mode,parent_folder,data_table
8,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20221108_Plate462_01,462,1,20221108,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
11,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20221108_Plate462_02,462,2,20221108,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
1,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20221108_Plate462_03,462,3,20221108,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...
5,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,20221108_Plate462_04,462,4,20221108,20.0,50.0,BF,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...,/gpfs/scratch1/shared/amftrackflow/Drp_downs/2...


In [55]:
run_parallel(
    "flux_extract.py",
    [directory_targ, 1],
    dataframe,
    4,
    "1:00:00",
    "flux_extract"
)

/home/svstaalduine/bash/job.sh
Sending jobs with id 1683212742448706482
4
Submitted batch job 2693822


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
